In [463]:
import nltk
from nltk import stem
snowball = stem.snowball.EnglishStemmer()

class Splitter(object):
    def __init__(self):
        self.nltk_splitter = nltk.data.load('tokenizers/punkt/english.pickle')
        self.nltk_tokenizer = nltk.tokenize.TreebankWordTokenizer()

    def split(self, text):
        """
        input format: a paragraph of text
        output format: a list of lists of words.
            e.g.: [['this', 'is', 'a', 'sentence'], ['this', 'is', 'another', 'one']]
        """
        sentences = self.nltk_splitter.tokenize(text)
        tokenized_sentences = [self.nltk_tokenizer.tokenize(sent) for sent in sentences]
        return tokenized_sentences


class POSTagger(object):
    def __init__(self):
        pass
        
    def pos_tag(self, sentences):
        """
        input format: list of lists of words
            e.g.: [['this', 'is', 'a', 'sentence'], ['this', 'is', 'another', 'one']]
        output format: list of lists of tagged tokens. Each tagged tokens has a
        form, a lemma, and a list of tags
            e.g: [[('this', 'this', ['DT']), ('is', 'be', ['VB']), ('a', 'a', ['DT']), ('sentence', 'sentence', ['NN'])],
                    [('this', 'this', ['DT']), ('is', 'be', ['VB']), ('another', 'another', ['DT']), ('one', 'one', ['CARD'])]]
        """

        pos = [nltk.pos_tag(sentence) for sentence in sentences]
        #adapt format
        pos = [[(word, word, [postag]) for (word, postag) in sentence] for sentence in pos]
        return pos

[-0.15625, 1.1607142857142858, 0.4166666666666667, 0.5514705882352942, -0.08928571428571429]

[-0.15625, 0.0, -0.05, -0.03676470588235294, -0.08928571428571429]

In [465]:
splitter = Splitter()
postagger = POSTagger()

In [466]:
splitted_sentences = splitter.split(text)
pos_tagged_sentences = postagger.pos_tag(splitted_sentences)

In [1]:
# lexicons = []
# with open("SentiWordNet.txt") as f:
#     for i in f.readlines():
#         lexicons.append(i)
# lexicons = lexicons[27:]

In [129]:
from collections import defaultdict, OrderedDict

def create_dict(text):
    pos_dict = defaultdict(int) 
    neg_dict = defaultdict(int)
    for i in range(len(text)):
        word = text[i].split("\t")[4].split("#")[0]
        if word == '':
            pass
        else:
            pos_dict[word] = float(text[i].split("\t")[2])
            neg_dict[word] = float(text[i].split("\t")[3])     
        
    return pos_dict, neg_dict

In [130]:
pos_dict = create_dict(lexicons)[0]
neg_dict = create_dict(lexicons)[1]

In [443]:
y= {"nicely": 0.25}
dict([(snowball.stem(k), v) for k, v in y.items()])

{u'nice': 0.25}

In [438]:
test_dict = dict([(snowball.stem(k), v) for k, v in pos_dict.items()])


In [444]:
test_dict_B = {snowball.stem(k): v for k, v in pos_dict.items()}


In [448]:
from collections import defaultdict

stem_words = defaultdict(list)

for k, v in pos_dict.items():
    stem_words[snowball.stem(k)].append((k, v))

stem_words['nice']

[(u'nicely', 0.25), (u'niceness', 0.625), (u'nice', 0.0)]

### Use sorted dictionaries?

In [449]:
#Use sorted dictionaries?
#pos_dict_sorted = OrderedDict(sorted(pos_dict.items(), key=lambda t: t[0]))
#neg_dict_sorted = OrderedDict(sorted(neg_dict.items(), key=lambda t: t[0]))

In [136]:
import json
json.dump(pos_dict, open("dicts/pos_dict", 'w'))
json.dump(neg_dict, open("dicts/neg_dict", 'w'))

pos_dict = json.load(open('dicts/pos_dict'))
neg_dict = json.load(open('dicts/neg_dict'))

In [476]:
import re

def tokenize(word):
    snowball = stem.snowball.EnglishStemmer()
    return snowball.stem(word.lower())

[0.0, 0.0, 0.0, 0.0, 0.0]


In [487]:
def cal_senti_score(pos_tagged_sentences):
    score = []
    
    for sent in pos_tagged_sentences:
        pos_score = []
        neg_score = []
        total = 0 

        total = len(sent)
        
        for token in sent:
            if pos_dict.get(token[0]) == None:
                pass
            else:
                #token[2].append(pos_dict[token[0]])
                pos_score.append(pos_dict[token[0]])
        for token in sent:
            if neg_dict.get(token[0]) == None:
                pass
            else:
                #token[2].append(neg_dict[token[0]])
                neg_score.append(neg_dict[token[0]])
        sent_score = (sum(pos_score) - sum(neg_score)) / float(total) 
        print pos_score
        print neg_score
        
        score.append(sent_score)
    return sum(score) / float(len(score))

In [494]:
cal_senti_score(pos_tagged_sentences)

[0.0, 0.0, 0.0]
[0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.625, 0.625]
[0.25, 0.375, 0.0, 0.0, 0.5]
[0.0, 0.5, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.375, 0.25, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.5, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


-0.009733893557422971

In [498]:
#print pos_dict['not']
print pos_dict['very']

0.25


In [464]:
text ="What can I say about this place. \
The staff of the restaurant is nice and the eggplant is not bad. \
Apart from that, very uninspired food, lack of atmosphere and too expensive. \
I am a staunch vegetarian and was sorely dissapointed with the veggie options on the menu. \
Will be the last time I visit, I recommend others to avoid."

Need to account for incrementers:
"too", "very", "sorely"
decrementers:
"barely", "little"
inverters:
"lack of"
"not"

Needs to remove stopwords from text, then tag the sentences?
Needs to stem the dictionary, but how?
